In [1]:
import sys
sys.path.append("..")

from typing import List, Dict, Callable, Optional
from datetime import datetime, timedelta
from IPython.display import display

import numpy as np
import pandas as pd

from sesto.constants import CURRENCIES, MT5Timeframe
import sesto.metatrader.data as mtd
from sesto.plot import plot_tradingview, plot_plotly
from sesto.backtester import Backtester, Trade

MetaTrader 5 initialized successfully.


In [2]:
# GENERATE ALL PAIR COMBINATIONS
from itertools import permutations

print(f"Number of currencies: {len(CURRENCIES)}")
# Generate all unique pairs of currency symbols
pairs = list(permutations(CURRENCIES, 2))
print(f"Number of pairs: {len(pairs)}")

Number of currencies: 8
Number of pairs: 56


In [3]:
# concatenate pairs into a single list
PAIRS = [pair[0] + pair[1] for pair in pairs]

In [4]:
TIMEFRAMES = [MT5Timeframe.M5, MT5Timeframe.M15, MT5Timeframe.H1, MT5Timeframe.H4]
DATA_FROM_DATE = datetime(2024, 4, 5)
DATA_TO_DATE = datetime(2024, 9, 19)

MA_PERIODS = [7, 14, 21]

CSM_PERIOD = 21

In [5]:
BACKTEST_FROM_DATE = datetime(2024, 1, 1)
BACKTEST_TO_DATE = datetime(2024, 9, 20)
MAIN_TIMEFRAME = MT5Timeframe.H1

INITIAL_CAPITAL = 10000

CSM_TREND_LONG_THRESHOLD = 65
CSM_TREND_SHORT_THRESHOLD = 35
CSM_REVERSAL_LONG_THRESHOLD = 90
CSM_REVERSAL_SHORT_THRESHOLD = 10
RSI_OVERSOLD = 30
RSI_OVERBOUGHT = 70
ATR_VOLATILITY_THRESHOLD = 0.02

CAPITAL_PER_TRADE = 200  # $200 per trade

RR_MULTIPLIER = 2
TRADE_SL_MULTIPLIER = 0.0025

SPREAD_MULTIPLIER = 0.0001
COST_MULTIPLIER = 0.0001
LEVERAGE = 500

In [6]:
mtd.fetch_data_all_timeframes(PAIRS, TIMEFRAMES, DATA_FROM_DATE, DATA_TO_DATE)

Failed to fetch data for symbol: USDEUR on timeframe: MT5Timeframe.M5
No data fetched for pair: USDEUR on timeframe: MT5Timeframe.M5
Failed to fetch data for symbol: USDGBP on timeframe: MT5Timeframe.M5
No data fetched for pair: USDGBP on timeframe: MT5Timeframe.M5
Fetched data for pair: USDJPY on timeframe: MT5Timeframe.M5
Fetched data for pair: USDCHF on timeframe: MT5Timeframe.M5
Fetched data for pair: USDCAD on timeframe: MT5Timeframe.M5
Failed to fetch data for symbol: USDAUD on timeframe: MT5Timeframe.M5
No data fetched for pair: USDAUD on timeframe: MT5Timeframe.M5
Failed to fetch data for symbol: USDNZD on timeframe: MT5Timeframe.M5
No data fetched for pair: USDNZD on timeframe: MT5Timeframe.M5
Fetched data for pair: EURUSD on timeframe: MT5Timeframe.M5
Fetched data for pair: EURGBP on timeframe: MT5Timeframe.M5
Fetched data for pair: EURJPY on timeframe: MT5Timeframe.M5
Fetched data for pair: EURCHF on timeframe: MT5Timeframe.M5
Fetched data for pair: EURCAD on timeframe: MT5T

In [7]:
# Remove symbols with no data for any timeframe
for timeframe in TIMEFRAMES:
    empty_symbols = [symbol for symbol, df in mtd.data[timeframe].items() if df is None or df.empty]
    for symbol in empty_symbols:
        del mtd.data[timeframe][symbol]
        if symbol in PAIRS:
            PAIRS.remove(symbol)

print(f"Remaining pairs after removing empty data: {len(PAIRS)}")

Remaining pairs after removing empty data: 28


In [8]:
import plotly.graph_objects as go

def plot_data_availability():
    # Calculate expected number of rows for each timeframe
    expected_rows = {}
    for timeframe in TIMEFRAMES:
        if timeframe == MT5Timeframe.M5:
            minutes_per_candle = 5
        elif timeframe == MT5Timeframe.M15:
            minutes_per_candle = 15
        elif timeframe == MT5Timeframe.H1:
            minutes_per_candle = 60
        elif timeframe == MT5Timeframe.H4:
            minutes_per_candle = 240
        
        total_minutes = (DATA_TO_DATE - DATA_FROM_DATE).total_seconds() / 60
        expected_rows[timeframe] = int(total_minutes / minutes_per_candle)

    # Create a DataFrame to store the normalized row counts
    heatmap_data = []

    for timeframe in TIMEFRAMES:
        for symbol in PAIRS:
            if symbol in mtd.data[timeframe]:
                actual_rows = len(mtd.data[timeframe][symbol])
                normalized_value = actual_rows / expected_rows[timeframe]
                heatmap_data.append({
                    'Timeframe': timeframe.name,
                    'Symbol': symbol,
                    'Normalized Row Count': normalized_value
                })

    heatmap_df = pd.DataFrame(heatmap_data)

    # Aggregate data to handle potential duplicates
    heatmap_df = heatmap_df.groupby(['Symbol', 'Timeframe'])['Normalized Row Count'].mean().reset_index()

    # Pivot the DataFrame for the heatmap
    heatmap_pivot = heatmap_df.pivot(index='Symbol', columns='Timeframe', values='Normalized Row Count')

    # Create the heatmap using Plotly with a dark theme
    fig = go.Figure(data=go.Heatmap(
        z=heatmap_pivot.values,
        x=heatmap_pivot.columns,
        y=heatmap_pivot.index,
        colorscale='Twilight',
        colorbar=dict(
            title='Normalized Row Count',
            titleside='right',
            thickness=18,
            tickfont=dict(color='white'),
            titlefont=dict(color='white')
        ),
        text=heatmap_pivot.values,
        texttemplate='%{text:.2f}',
        textfont={"size": 12, "color": "black"},
    ))

    # Update layout with dark theme and improved spacing
    fig.update_layout(
        title={
            'text': 'Normalized Row Count Heatmap: Actual vs Expected',
            'y': 0.95,
            'x': 0.5,
            'xanchor': 'center',
            'yanchor': 'top'
        },
        yaxis_title='Symbol',
        height=max(600, len(PAIRS) * 25),  # Minimum height of 600px
        width=900,
        paper_bgcolor='rgb(30,30,30)',
        plot_bgcolor='rgb(30,30,30)',
        font=dict(color='white', family="Inter"),
        title_font=dict(color='white', size=16),
        xaxis=dict(
            tickfont=dict(color='white', size=12),
            title_font=dict(color='white', size=16),
            side='top',
            tickangle=-45
        ),
        yaxis=dict(
            tickfont=dict(color='white', size=12),
            title_font=dict(color='white', size=16),
            automargin=True,
        ),
        margin=dict(l=125, r=50, t=100, b=50)
    )

    # Show the plot
    fig.show()

# plot_data_availability()

In [9]:
from sesto.indicators import SMA, EMA, RSI, ROC, MACD, BB, ATR

for timeframe, pairs_data in mtd.data.items():
    for pair, df in pairs_data.items():
        if df is not None and not df.empty:
            for period in MA_PERIODS:
                SMA(df, period)
                EMA(df, period)
                ROC(df, period)
                RSI(df, period)
                BB(df, period, 2)
                ATR(df, period)
    
            df['psm'] = df[f'rsi-{CSM_PERIOD}'] + df[f'roc-{CSM_PERIOD}'] / 2

display(mtd.data[MT5Timeframe.H1]['EURUSD'].head())



,time,open,high,low,close,tick_volume,spread,real_volume,sma-7,ema-7,...,bbu-14-2,bbl-14-2,sma-21,ema-21,roc-21,rsi-21,bbm-21-2,bbu-21-2,bbl-21-2,psm
0,2024-04-04 21:00:00,1.08599,1.08599,1.08465,1.08499,2881,2,0,NaN,1.084990,...,NaN,NaN,NaN,1.084990,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-04-04 22:00:00,1.08498,1.08506,1.08332,1.08335,3437,1,0,NaN,1.084580,...,NaN,NaN,NaN,1.084841,NaN,NaN,NaN,NaN,NaN,NaN
2,2024-04-04 23:00:00,1.08335,1.08383,1.08316,1.08377,865,2,0,NaN,1.084377,...,NaN,NaN,NaN,1.084744,NaN,NaN,NaN,NaN,NaN,NaN
3,2024-04-05 00:00:00,1.08350,1.08399,1.08337,1.08375,1598,13,0,NaN,1.084221,...,NaN,NaN,NaN,1.084653,NaN,NaN,NaN,NaN,NaN,NaN
4,2024-04-05 01:00:00,1.08369,1.08398,1.08354,1.08362,659,4,0,NaN,1.084070,...,NaN,NaN,NaN,1.084559,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
import plotly.graph_objects as go
import pandas as pd

def plot_indicator_availability():
    # Calculate expected number of rows for each timeframe
    expected_rows = {}
    for timeframe in TIMEFRAMES:
        if timeframe == MT5Timeframe.M5:
            minutes_per_candle = 5
        elif timeframe == MT5Timeframe.M15:
            minutes_per_candle = 15
        elif timeframe == MT5Timeframe.H1:
            minutes_per_candle = 60
        elif timeframe == MT5Timeframe.H4:
            minutes_per_candle = 240
        
        total_minutes = (DATA_TO_DATE - DATA_FROM_DATE).total_seconds() / 60
        expected_rows[timeframe] = int(total_minutes / minutes_per_candle)

    # Create a DataFrame to store the normalized row counts
    heatmap_data = []

    for timeframe in TIMEFRAMES:
        for symbol in PAIRS:
            if symbol in mtd.data[timeframe]:
                df = mtd.data[timeframe][symbol]
                if df is not None and not df.empty:
                    # Check for the presence of indicators
                    indicator_columns = [col for col in df.columns if col.startswith(('sma-', 'ema-', 'rsi-', 'roc-', 'macd', 'bb'))]
                    if indicator_columns:
                        actual_rows = df[indicator_columns].dropna().shape[0]
                        normalized_value = actual_rows / expected_rows[timeframe]
                        heatmap_data.append({
                            'Timeframe': timeframe.name,
                            'Symbol': symbol,
                            'Normalized Indicator Count': normalized_value
                        })

    heatmap_df = pd.DataFrame(heatmap_data)

    # Aggregate data to handle potential duplicates
    heatmap_df = heatmap_df.groupby(['Symbol', 'Timeframe'])['Normalized Indicator Count'].mean().reset_index()

    # Pivot the DataFrame for the heatmap
    heatmap_pivot = heatmap_df.pivot(index='Symbol', columns='Timeframe', values='Normalized Indicator Count')

    # Create the heatmap using Plotly with a dark theme
    fig = go.Figure(data=go.Heatmap(
        z=heatmap_pivot.values,
        x=heatmap_pivot.columns,
        y=heatmap_pivot.index,
        colorscale='Twilight',
        colorbar=dict(
            title='Normalized Indicator Count',
            titleside='right',
            thickness=18,
            tickfont=dict(color='white'),
            titlefont=dict(color='white')
        ),
        text=heatmap_pivot.values,
        texttemplate='%{text:.2f}',
        textfont={"size": 12, "color": "black"},
    ))

    # Update layout with dark theme and improved spacing
    fig.update_layout(
        title={
            'text': 'Normalized Indicator Count Heatmap: Actual vs Expected',
            'y': 0.95,
            'x': 0.5,
            'xanchor': 'center',
            'yanchor': 'top'
        },
        yaxis_title='Symbol',
        height=max(600, len(PAIRS) * 25),  # Minimum height of 600px
        width=900,
        paper_bgcolor='rgb(30,30,30)',
        plot_bgcolor='rgb(30,30,30)',
        font=dict(color='white', family="Inter"),
        title_font=dict(color='white', size=16),
        xaxis=dict(
            tickfont=dict(color='white', size=12),
            title_font=dict(color='white', size=16),
            side='top',
            tickangle=-45
        ),
        yaxis=dict(
            tickfont=dict(color='white', size=12),
            title_font=dict(color='white', size=16),
            automargin=True,
        ),
        margin=dict(l=125, r=50, t=100, b=50)
    )

    # Show the plot
    fig.show()

# plot_indicator_availability()

In [11]:
import pandas_ta as ta

for timeframe in TIMEFRAMES:
    for currency in CURRENCIES:
        # Initialize DataFrames to hold PSM values for the current currency as base and quote
        base_psm_df = pd.DataFrame()
        quote_psm_df = pd.DataFrame()
        
        # Iterate over all pair data for the current timeframe
        for symbol, df in mtd.data[timeframe].items():
            if df is None or df.empty:
                continue
            
            base_currency = symbol[:3]
            quote_currency = symbol[3:]
            
            if currency == base_currency:
                # Use PSM as is for base currency
                base_psm_df[symbol] = df['psm']
                base_psm_df['time'] = df['time']
            elif currency == quote_currency:
                # Invert PSM for quote currency
                quote_psm_df[symbol] = -df['psm']
                quote_psm_df['time'] = df['time']
        
        # Calculate base CSM
        if not base_psm_df.empty:
            base_csm_df = base_psm_df.groupby('time').mean()
            base_csm_df['csm'] = base_csm_df.mean(axis=1)
            base_csm_df['csm'] = base_csm_df['csm'].rolling(CSM_PERIOD).mean()
            min_csm = base_csm_df['csm'].min()
            max_csm = base_csm_df['csm'].max()
            base_csm_df['csm'] = 100 * (base_csm_df['csm'] - min_csm) / (max_csm - min_csm)
            
            # Calculate MACD for base CSM
            MACD(base_csm_df, 'csm', 12, 26, 9)
        
        # Calculate quote CSM
        if not quote_psm_df.empty:
            quote_csm_df = quote_psm_df.groupby('time').mean()
            quote_csm_df['csm'] = quote_csm_df.mean(axis=1)
            quote_csm_df['csm'] = quote_csm_df['csm'].rolling(CSM_PERIOD).mean()
            min_csm = quote_csm_df['csm'].min()
            max_csm = quote_csm_df['csm'].max()
            quote_csm_df['csm'] = 100 * (quote_csm_df['csm'] - min_csm) / (max_csm - min_csm)
            
            # Calculate MACD for quote CSM
            MACD(quote_csm_df, 'csm', 12, 26, 9)
        
        # Add CSM and MACD values to all relevant symbols in mtd.data
        for symbol in mtd.data[timeframe]:
            base_currency = symbol[:3]
            quote_currency = symbol[3:]
            
            if currency == base_currency and not base_csm_df.empty:
                mtd.data[timeframe][symbol]['base_csm'] = mtd.data[timeframe][symbol]['time'].map(base_csm_df['csm'])
                mtd.data[timeframe][symbol]['base_csm_macd'] = mtd.data[timeframe][symbol]['time'].map(base_csm_df['macd'])
                mtd.data[timeframe][symbol]['base_csm_macd_signal'] = mtd.data[timeframe][symbol]['time'].map(base_csm_df['macd-signal'])
                mtd.data[timeframe][symbol]['base_csm_macd_histogram'] = mtd.data[timeframe][symbol]['time'].map(base_csm_df['macd-histogram'])
                mtd.data[timeframe][symbol]['base_csm_rsi'] = ta.rsi(mtd.data[timeframe][symbol]['base_csm'], length=CSM_PERIOD)
                mtd.data[timeframe][symbol]['base_csm_roc'] = ta.roc(mtd.data[timeframe][symbol]['base_csm'], length=CSM_PERIOD)

            elif currency == quote_currency and not quote_csm_df.empty:

                mtd.data[timeframe][symbol]['quote_csm'] = mtd.data[timeframe][symbol]['time'].map(quote_csm_df['csm'])
                mtd.data[timeframe][symbol]['quote_csm_macd'] = mtd.data[timeframe][symbol]['time'].map(quote_csm_df['macd'])
                mtd.data[timeframe][symbol]['quote_csm_macd_signal'] = mtd.data[timeframe][symbol]['time'].map(quote_csm_df['macd-signal'])
                mtd.data[timeframe][symbol]['quote_csm_macd_histogram'] = mtd.data[timeframe][symbol]['time'].map(quote_csm_df['macd-histogram'])
                mtd.data[timeframe][symbol]['quote_csm_rsi'] = ta.rsi(mtd.data[timeframe][symbol]['quote_csm'], length=CSM_PERIOD)
                mtd.data[timeframe][symbol]['quote_csm_roc'] = ta.roc(mtd.data[timeframe][symbol]['quote_csm'], length=CSM_PERIOD)



In [12]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import random

def plot_random_data():
    # Choose a random timeframe and symbol
    random_timeframe = random.choice(TIMEFRAMES)
    random_symbol = random.choice(PAIRS)

    # Get the data for the chosen timeframe and symbol
    df = mtd.data[random_timeframe][random_symbol]

    # Create subplots
    fig = make_subplots(rows=6, cols=1, shared_xaxes=True, vertical_spacing=0.05,
                        subplot_titles=('Close Price + SMA', 'CSM Values', 'Base CSM MACD', 'Quote CSM MACD', 'RSI', 'ROC'))

    # 1. Close Price + SMA
    fig.add_trace(go.Scatter(x=df['time'], y=df['close'], name='Close Price'), row=1, col=1)
    fig.add_trace(go.Scatter(x=df['time'], y=df['sma-14'], name='SMA-14'), row=1, col=1)

    # 2. CSM Values
    fig.add_trace(go.Scatter(x=df['time'], y=df['base_csm'], name='Base CSM'), row=2, col=1)
    fig.add_trace(go.Scatter(x=df['time'], y=df['quote_csm'], name='Quote CSM'), row=2, col=1)

    # 3. Base CSM MACD
    fig.add_trace(go.Scatter(x=df['time'], y=df['base_csm_macd'], name='Base MACD'), row=3, col=1)
    fig.add_trace(go.Scatter(x=df['time'], y=df['base_csm_macd_signal'], name='Base Signal'), row=3, col=1)
    fig.add_bar(x=df['time'], y=df['base_csm_macd_histogram'], name='Base Histogram', row=3, col=1)

    # 4. Quote CSM MACD
    fig.add_trace(go.Scatter(x=df['time'], y=df['quote_csm_macd'], name='Quote MACD'), row=4, col=1)
    fig.add_trace(go.Scatter(x=df['time'], y=df['quote_csm_macd_signal'], name='Quote Signal'), row=4, col=1)
    fig.add_bar(x=df['time'], y=df['quote_csm_macd_histogram'], name='Quote Histogram', row=4, col=1)

    # 5. RSI
    fig.add_trace(go.Scatter(x=df['time'], y=df['rsi-14'], name='RSI-14'), row=5, col=1)

    # 6. ROC
    fig.add_trace(go.Scatter(x=df['time'], y=df['roc-14'], name='ROC-14'), row=6, col=1)

    # Update layout
    fig.update_layout(
        height=1500, 
        width=1600, 
        title_text=f"Chart for {random_symbol} ({random_timeframe.name})", 
        template='plotly_dark'
    )

    # Update x-axis to include range sl_pider
    fig.update_xaxes(
        rangesl_pider_visible=True,
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(count=1, label="YTD", step="year", stepmode="todate"),
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(step="all")
            ])
        ),
        row=6, col=1  # Apply to the bottom subplot
    )

    # Hide rangesl_pider for all subplots except the bottom one
    for i in range(1, 6):
        fig.update_xaxes(rangesl_pider_visible=False, row=i, col=1)

    fig.update_yaxes(title_text="Price", row=1, col=1)
    fig.update_yaxes(title_text="CSM", row=2, col=1)
    fig.update_yaxes(title_text="Base MACD", row=3, col=1)
    fig.update_yaxes(title_text="Quote MACD", row=4, col=1)
    fig.update_yaxes(title_text="RSI", row=5, col=1)
    fig.update_yaxes(title_text="ROC", row=6, col=1)

    # Show the plot
    fig.show()

# plot_random_data()

In [13]:
class MyStrategy(Backtester):    
    def entry_condition(
        self, 
        symbol: str, 
        time: datetime, 
        row: pd.Series, 
        open_trades: List[Trade], 
        closed_trades: List[Trade], 
        timeframe: MT5Timeframe,
    ) -> Optional[Dict]:
        # Check if we already have an open trade for this symbol
        if any(trade.symbol == symbol for trade in open_trades):
            return None

        base_csm = row.get('base_csm')
        quote_csm = row.get('quote_csm')
        rsi = row.get('rsi-14')
    
        trend_up = (
            base_csm > CSM_TREND_LONG_THRESHOLD and
            base_csm < CSM_REVERSAL_LONG_THRESHOLD and
            quote_csm < CSM_TREND_SHORT_THRESHOLD and
            quote_csm > CSM_REVERSAL_SHORT_THRESHOLD
        )

        trend_down = (
            base_csm < CSM_TREND_SHORT_THRESHOLD and 
            base_csm > CSM_REVERSAL_LONG_THRESHOLD and
            quote_csm > CSM_TREND_LONG_THRESHOLD and
            quote_csm < CSM_REVERSAL_SHORT_THRESHOLD
        )

        entry = None
        entry_p = row['close']

        if trend_up:
            print(f"{time} - {symbol} - ENTRY CONDITION - TREND_UP - CLOSE: ${row['close']:.3f} - BASE_CSM: {base_csm:.3f} - QUOTE_CSM: {quote_csm:.3f} - RSI: {rsi:.3f}")
            entry = {
                'entry_p': entry_p,
                'type': 'long',
                'capital_allocation': CAPITAL_PER_TRADE,
                'tp_p': entry_p * (1 + (TRADE_SL_MULTIPLIER * RR_MULTIPLIER)),
                'sl_p': entry_p * (1 - TRADE_SL_MULTIPLIER)
            }
        elif trend_down:
            print(f"{time} - {symbol} - ENTRY CONDITION - TREND_DOWN - CLOSE: ${row['close']:.3f} - BASE_CSM: {base_csm:.3f} - QUOTE_CSM: {quote_csm:.3f} - RSI: {rsi:.3f}")
            entry = {
                'entry_p': entry_p,
                'type': 'short',
                'capital_allocation': CAPITAL_PER_TRADE,
                'tp_p': entry_p * (1 - (TRADE_SL_MULTIPLIER * RR_MULTIPLIER)),
                'sl_p': entry_p * (1 + TRADE_SL_MULTIPLIER)
            }

        # Ensure no open trades for the symbol
        if entry and not any(trade.symbol == symbol for trade in open_trades):
            return entry

        return None

    def exit_condition(
        self, 
        trade: Trade, 
        time: datetime, 
        row: pd.Series, 
        open_trades: List[Trade], 
        closed_trades: List[Trade], 
        timeframe: MT5Timeframe
    ) -> bool:
        return False

    def trailing_stop(
        self, 
        trade: Trade, 
        time: datetime, 
        row: pd.Series, 
        open_trades: List[Trade], 
        closed_trades: List[Trade], 
        timeframe: MT5Timeframe
    ):
        """
        Implement a trailing stop that adjusts the stop loss to make the trade risk-free
        once the unrealized PnL exceeds 5% of the capital allocation.
        """
        # Define the threshold for trailing stop activation
        TRAILING_STOP_BREAKEVEN_PNL_THRESHOLD = 0.05 * trade.used_capital  # 5% of capital allocation
        TRAILING_STOP_CASINO_MONEY_THRESHOLD = 0.5 * trade.used_capital  # 50% of capital allocation

        # Check if unrealized PnL exceeds the threshold
        if trade.unrealized_pnl > TRAILING_STOP_CASINO_MONEY_THRESHOLD:
            old_metrics = {
                'sl_p': trade.sl_p,
                'potential_loss_usd': trade.potential_loss_usd,
                'potential_loss_percent': trade.potential_loss_percent,
                'potential_profit_usd': trade.potential_profit_usd,
                'potential_profit_percent': trade.potential_profit_percent
            }
            
            if trade.type == 'long':
                trade.sl_p = trade.entry_p * (1 + TRADE_SL_MULTIPLIER)
            else:
                trade.sl_p = trade.entry_p * (1 - TRADE_SL_MULTIPLIER)
            
            trade.calculate_potential_outcomes()

            new_metrics = {
                'sl_p': trade.sl_p,
                'potential_loss_usd': trade.potential_loss_usd,
                'potential_loss_percent': trade.potential_loss_percent,
                'potential_profit_usd': trade.potential_profit_usd,
                'potential_profit_percent': trade.potential_profit_percent
            }

            print(f"{row['time']} - {trade.symbol} - TRAILING STOP - UNREALIZED PNL: ${trade.unrealized_pnl:.3f} - OLD SL: {old_metrics['sl_p']:.3f} - NEW SL: ${trade.sl_p:.3f} - OLD LOSS AT SL: ${old_metrics['potential_loss_usd']:.3f} - NEW LOSS AT SL: ${trade.potential_loss_usd:.3f}")
        
        elif trade.unrealized_pnl > TRAILING_STOP_BREAKEVEN_PNL_THRESHOLD and (trade.type == 'long' and row['close'] > trade.be_p or trade.type == 'short' and row['close'] < trade.be_p):
            # Check if trailing stop has not been set yet
            if trade.potential_loss_usd > 0 or trade.potential_loss_percent > 0:
                old_metrics = {
                    'sl_p': trade.sl_p,
                    'potential_loss_usd': trade.potential_loss_usd,
                    'potential_loss_percent': trade.potential_loss_percent,
                    'potential_profit_usd': trade.potential_profit_usd,
                    'potential_profit_percent': trade.potential_profit_percent
                }

                # Update the trade's stop loss
                trade.sl_p = trade.be_p

                # Update potential loss to reflect the new stop loss (should be zero)
                trade.calculate_potential_outcomes()

                new_metrics = {
                    'sl_p': trade.sl_p,
                    'potential_loss_usd': trade.potential_loss_usd,
                    'potential_loss_percent': trade.potential_loss_percent,
                    'potential_profit_usd': trade.potential_profit_usd,
                    'potential_profit_percent': trade.potential_profit_percent
                }

                # Create DataFrame to compare old and new metrics
                comparison_df = pd.DataFrame({
                    'Metric': old_metrics.keys(),
                    'Old Value': old_metrics.values(),
                    'New Value': new_metrics.values()
                })

                print(f"{row['time']} - {trade.symbol} - TRAILING STOP - UNREALIZED PNL: ${trade.unrealized_pnl:.3f} - OLD SL: {old_metrics['sl_p']:.3f} - NEW SL: ${trade.sl_p:.3f} - OLD LOSS AT SL: ${old_metrics['potential_loss_usd']:.3f} - NEW LOSS AT SL: ${trade.potential_loss_usd:.3f}")
                # display(comparison_df)




In [14]:
backtest = MyStrategy(
    mtd.data, 
    initial_capital=INITIAL_CAPITAL, 
    main_timeframe=MAIN_TIMEFRAME,
    cost_multiplier=COST_MULTIPLIER, 
    spread_multiplier=SPREAD_MULTIPLIER,
    leverage=LEVERAGE
)

backtest.run()

2024-04-25 10:00:00 - USDJPY - ENTRY CONDITION - TREND_UP - CLOSE: $155.626 - BASE_CSM: 80.345 - QUOTE_CSM: 33.462
2024-04-25 10:00:00 - USDJPY - OPENED TRADE - long - ENTRY: $155.642 - TP: $156.388 - SL: $155.252 - LIQ: $155.330 - BE: $155.647 - AVAILABLE CAPITAL: $9796.8
2024-04-25 11:00:00 - USDJPY - TRAILING STOP - UNREALIZED PNL: $15.714 - OLD SL: 155.252 - NEW SL: $155.647 - OLD LOSS AT SL: $246.800 - NEW LOSS AT SL: $-6.400
2024-04-25 12:00:00 - USDJPY - CLOSED TRADE - long - ENTRY: $155.642 - CLOSE: $155.647 - PNL: $-10.00 - SL: $155.647 - TP: $156.388 - REASON: SL - AVAILABLE CAPITAL: $9986.79967999999
2024-04-25 12:00:00 - USDJPY - ENTRY CONDITION - TREND_UP - CLOSE: $155.631 - BASE_CSM: 78.083 - QUOTE_CSM: 29.172
2024-04-25 12:00:00 - USDJPY - OPENED TRADE - long - ENTRY: $155.647 - TP: $156.394 - SL: $155.257 - LIQ: $155.335 - BE: $155.652 - AVAILABLE CAPITAL: $9783.599679999988
2024-04-25 15:00:00 - USDJPY - TRAILING STOP - UNREALIZED PNL: $16.998 - OLD SL: 155.257 - NEW S

In [15]:
trade_log = pd.DataFrame(backtest.trade_log)
trade_log[['type', 'entry_p', 'tp_p', 'sl_p', 'liq_p','pnl', 'potential_loss_usd', 'potential_loss_percent', 'sl_price_diff_percent']]
# trade_log.to_csv('./log.csv')

,type,entry_p,tp_p,sl_p,liq_p,pnl,potential_loss_usd,potential_loss_percent,sl_price_diff_percent
0,long,155.641563,156.388490,155.646543,155.330279,-10.000320,-6.4,-3.2,-3.2
1,long,155.646563,156.393514,155.651544,155.335270,-10.000320,-6.4,-3.2,-3.2
2,long,155.664565,156.411602,156.053726,155.353236,466.654020,-253.2,-126.6,-126.6
3,long,155.561555,156.308098,155.566533,155.250431,-10.000320,-6.4,-3.2,-3.2
4,long,155.504549,156.250818,155.509525,155.193540,-10.000320,-6.4,-3.2,-3.2
...,...,...,...,...,...,...,...,...,...
1420,long,0.840694,0.844729,0.840721,0.839013,-10.000320,-6.4,-3.2,-3.2
1421,long,168.121810,168.928630,167.701506,167.785567,-97.541418,246.8,123.4,123.4
1422,long,104.317431,104.818052,104.578224,104.108796,108.118788,-253.2,-126.6,-126.6
1423,long,96.099609,96.560793,95.859360,95.907410,-85.626974,246.8,123.4,123.4


In [16]:
performance_report = backtest.generate_report()
performance_report
# performance_report.to_csv('./performance_report.csv')

,Metric,Value
0,Initial Capital,$10000.00
1,Final Capital,$95929.83
2,Total Profit,$85929.83
3,Return (%),859.30%
4,Annualized Return (%),15707.14%
5,Volatility (Ann.),246.52
6,Sharpe Ratio,63.71
7,Sortino Ratio,196.65
8,Calmar Ratio,1161.24
9,Max. Drawdown ($),$-1352.62
